Finetuning BERT for text classification: classifying phishing urls


In [ ]:
!pip install datasets
!pip install evaluate
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00


In [ ]:
#lib
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

In [ ]:
#load dataset
dataset = load_dataset("shawhin/phishing-site-classification")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/98.0k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/24.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/450 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/450 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2100
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
})

In [ ]:
#load bert pretrained model
model_name = "google-bert/bert-base-uncased"
#load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
#load model with binary classification head
id2label = {0: "Safe" , 1:"Not Safe"}
label2id = {"Safe": 0 , "Not Safe" : 1}
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2, id2label = id2label, label2id = label2id,)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#freeze all parameters
for name, param in model.base_model.named_parameters():
  param.requires_grad = False

#unfreeze final layer and classification head
for name, param in model.base_model.named_parameters():
  if "pooler" in name:
    param.requires_grad = True

In [ ]:
#preprocess data
def preprocess_data(examples):
  return tokenizer(examples['text'] , truncation = True)

tokenized_dataset = dataset.map(preprocess_data ,batched = True)

Map:   0%|          | 0/2100 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [ ]:
#datacollator
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [ ]:
#metrics
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

In [ ]:
def compute_metrics(eval_pred):
  #get predictions
  predictions, labels = eval_pred

  #apply softmax to get probabilities
  probabilities = np.exp(predictions) / np.exp(predictions).sum(-1,
                                                                keepdims=True)
  #use probabilities of the positive class for ROC AUC
  positive_class_probs = probabilities[:, 1]
  #compute auc
  auc = np.round(auc_score.compute(prediction_scores=positive_class_probs,
                                    references=labels)['roc_auc'],3)

  #predict most probable class
  predicted_classes = np.argmax(predictions, axis=1)
  #compute accuracy
  acc = np.round(accuracy.compute(predictions=predicted_classes,
                                    references=labels)['accuracy'],3)

  return {"Accuracy": acc, "AUC": auc}

In [ ]:
#hyperparameters
lr = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir="bert-phishing-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
#trainer class
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()

<ipython-input-12-517f883c7f7a>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.504900,0.382729,0.816000,0.913000
2,0.410600,0.338188,0.836000,0.932000
3,0.355800,0.313868,0.858000,0.939000
4,0.356400,0.358055,0.842000,0.945000
5,0.350400,0.335082,0.867000,0.948000
6,0.347900,0.290868,0.869000,0.950000
7,0.334400,0.289739,0.878000,0.950000
8,0.311500,0.289415,0.869000,0.950000
9,0.312900,0.285038,0.871000,0.951000
10,0.313800,0.289961,0.867000,0.951000


TrainOutput(global_step=2630, training_loss=0.3598565772459081, metrics={'train_runtime': 544.3862, 'train_samples_per_second': 38.576, 'train_steps_per_second': 4.831, 'total_flos': 706603239165360.0, 'train_loss': 0.3598565772459081, 'epoch': 10.0})

In [ ]:
# apply model to validation dataset
predictions = trainer.predict(tokenized_dataset["validation"])

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
metrics = compute_metrics((logits, labels))
print(metrics)

{'Accuracy': 0.889, 'AUC': 0.945}
